# ANALIZA I PRIMENA ALATA WEIGHTS & BIASES (W&B) U PRAĆENJU ML MODELA

### Uvod

U razvoju modela mašinskog učenja, posebno u eksperimentalnim fazama, neophodno je sistematsko praćenje parametara, metrika i verzija modela. 
Ručno praćenje eksperimenata brzo postaje neefikasno i nepouzdano.

Alat Weights & Biases (W&B) predstavlja platformu za: <br>
- praćenje eksperimenata (experiment tracking) <br>
- logovanje metrika u realnom vremenu <br>
- verzionisanje modela i podataka <br>
- vizuelizaciju trening procesa <br>
- poređenje više modela <br>

U ovom radu prikazujemo:
  
1. Proceduru instalacije i podešavanja W&B <br>
2. Povezivanje sa modelima za predikciju hepatitisa C <br>
3. Analizu funkcionalnosti <br> 
4. Poređenje sa alatom MLflow <br>

### Setup i instalacija Weights & Biases

#### Kreiranje naloga

1. Otvara se nalog na platformi https://wandb.ai<br>
2. Generiše se API ključ<br>
3. API ključ se koristi za autentifikaciju projekta

#### Instalacija

Instalirana je biblioteka putem sledeće komande

--pip install wandb nbformat scikit-learn pandas


#### Autentifikacija

Nakon instalacije:

Povezivanje sa nalogom 

In [ ]:
import wandb
wandb.login()


True

Unutar funkcije run_and_log, koristi se wandb.init() koja uspostavlja vezu izmedju lokalne skripte i cloud-a. 
Paramtar project grupiše sve eksperimente (run-ove) vezane za predikciju Hepatitisa C na jedno mesto, čime se omogućava automatsko generisanje tabela i vizuelizacija. 
Parametar entity definiše vlasnika projekta (korisnički nalog ili tim), osiguravajući centralizovano skladištenje podataka i omogućavajući saradnju više istraživača unutar istog radnog prostora.

Svi hiperparametri (npr. n_estimators, max_depth) su spakovani u wandb.config rečnik kako bi se omogućila kasnija analiza uticaja parametara na uspeh modela.
Testirana su tri tipa algoritma: Random Forest, Logistička regresija i Decision Tree. Za svaki model su kreirane po dve varijante sa različitim hiperparametrima kako bi se iskoristile W&B funkcije za poređenje.

Nakon faze testiranja, komandom wandb.log() zabeleženi su ključne metrike: Accuracy, Precision, Recall i F1-score, kao i matrica konfuzije (Confusion Matrix) prikazana kao wandb.Table.

In [1]:
import wandb
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import IterativeImputer

#  wandb.login()

df = pd.read_csv("HepatitisCdata.csv")

# X kolonu prikazuje kao Unnamed zato je izbacujemo
if "Unnamed: 0" in df.columns:
    df = df.drop("Unnamed: 0", axis=1)

# Prebacivanje m/f u 0/1, faktorisanje
df["Sex"] = df["Sex"].astype("category").cat.codes

# =========================
#Binarizacija
# 0 = Blood Donor
# 1 = Hepatitis C
df["Category"] = df["Category"].apply(lambda x: 0 if "Blood Donor" in str(x) else 1)

X = df.drop("Category", axis=1)
y = df["Category"]

# Brisanje NA vrednosti
#imputer = SimpleImputer(strategy="median")
#X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

# MICE
mice_imputer = IterativeImputer(random_state=42)
X_imputed = mice_imputer.fit_transform(X)

X = pd.DataFrame(X_imputed, columns=X.columns)

features = list(X.columns)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)

print("Train distribution:\n", y_train.value_counts())
print("Test distribution:\n", y_test.value_counts())

def run_and_log(model, run_name, config, scale_for_lr=False):
    run = wandb.init(
        project="hepatitis_c_prediction",
        entity="milosandjelkovic20-faculty-of-natural-sciences-kragujevac",
        name=run_name,
        config={**config, "features": features},
        reinit=True
    )

    # Skaliranje
    if scale_for_lr:
        scaler = StandardScaler()
        X_tr = scaler.fit_transform(X_train)
        X_te = scaler.transform(X_test)
    else:
        X_tr = X_train.values
        X_te = X_test.values

    model.fit(X_tr, y_train)
    pred = model.predict(X_te)

    y_true = y_test.to_numpy()
    y_pred = np.asarray(pred)

    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, zero_division=0)
    rec  = recall_score(y_true, y_pred, zero_division=0)
    f1   = f1_score(y_true, y_pred, zero_division=0)

    cm = confusion_matrix(y_true, y_pred, labels=[0, 1])

    wandb.log({
        "accuracy": acc,
        "precision": prec,
        "recall": rec,
        "f1_score": f1,
        "confusion_matrix": wandb.Table(
            data=cm.tolist(),
            columns=["Pred_BloodDonor", "Pred_HepatitisC"]
        )
    })

    wandb.finish()

#RandomForest
run_and_log(
    RandomForestClassifier(n_estimators=200, random_state=42),
    run_name="RandomForest-baseline",
    config={"model": "RandomForest", "n_estimators": 200, "test_size": 0.20, "random_state": 42}
)

# RandomForest - Model 2
run_and_log(
    RandomForestClassifier(n_estimators=50, max_depth=10, random_state=42),
    run_name="RandomForest-variant-2",
    config={"model": "RandomForest", "n_estimators": 50, "max_depth": 10, "test_size": 0.20}
)

#LogisticRegression
run_and_log(
    LogisticRegression(max_iter=2000),
    run_name="LogisticRegression-baseline",
    config={"model": "LogisticRegression", "max_iter": 2000, "test_size": 0.20, "random_state": 42},
    scale_for_lr=True
)

# LogisticRegression - Model 2
run_and_log(
    LogisticRegression(max_iter=2000, C=0.1), 
    run_name="LogisticRegression-variant-2",
    config={"model": "LogisticRegression", "max_iter": 2000, "C": 0.1, "test_size": 0.20},
    scale_for_lr=True
)

#DecisionTree
run_and_log(
    DecisionTreeClassifier(random_state=42, max_depth=5),
    run_name="DecisionTree-baseline",
    config={"model": "DecisionTree", "max_depth": 5, "test_size": 0.20, "random_state": 42}
)

# DecisionTree - Model 2
run_and_log(
    DecisionTreeClassifier(random_state=42, max_depth=3),
    run_name="DecisionTree-variant-2",
    config={"model": "DecisionTree", "max_depth": 3, "test_size": 0.20}
)


wandb: [wandb.login()] Loaded credentials for https://api.wandb.ai from C:\Users\PC\_netrc.


Train distribution:
 Category
0    432
1     60
Name: count, dtype: int64
Test distribution:
 Category
0    108
1     15
Name: count, dtype: int64


wandb: Currently logged in as: milosandjelkovic20 (milosandjelkovic20-faculty-of-natural-sciences-kragujevac) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


accuracy,▁
f1_score,▁
precision,▁
recall,▁
accuracy,0.97561
f1_score,0.88889
precision,1
recall,0.8


accuracy,▁
f1_score,▁
precision,▁
recall,▁
accuracy,0.95935
f1_score,0.81481
precision,0.91667
recall,0.73333


accuracy,▁
f1_score,▁
precision,▁
recall,▁
accuracy,0.95122
f1_score,0.75
precision,1
recall,0.6


accuracy,▁
f1_score,▁
precision,▁
recall,▁
accuracy,0.95122
f1_score,0.75
precision,1
recall,0.6


accuracy,▁
f1_score,▁
precision,▁
recall,▁
accuracy,0.97561
f1_score,0.90323
precision,0.875
recall,0.93333


accuracy,▁
f1_score,▁
precision,▁
recall,▁
accuracy,0.95122
f1_score,0.78571
precision,0.84615
recall,0.73333


<p align="center">
  <img src="wandb-runs-tab.png" width="600"/>
</p>

<p align="center">
Slika 1: Lista eksperimenata u okviru W&B projekta hepatitis_c_prediction
</p>

---

<p align="center">
  <img src="wandb-metrike-modela.png" width="600"/>
</p>

<p align="center">
Slika 2: Prikaz metrika svih modela
</p>

---

<p align="center">
  <img src="wandb-overview-panel.png" width="600"/>
</p>

<p align="center">
Slika 3: Detaljan prikaz RandomForest-baseline run-a sa logovanim hiperparametrima i metrikama
</p>

---

<p align="center">
  <img src="wandb-metrika-modela.png" width="600"/>
</p>

<p align="center">
Slika 4: Grafički prikaz vrednosti metrika
</p>


### Analiza rezultata

U okviru projekta uspešno je demonstrirana integracija modela za klasifikaciju hepatitisa C sa platformom Weights & Biases, koji omogućava automatsko logovanje hiperparametara i metrika, kao i grafički prikaz performansi modela. 
Analizom dobijenih rezultata utvrđeno je da izbor algoritma ima primarni uticaj na preciznost analize, pri čemu je Random Forest pokazao najveću otpornost na disbalans klasa i postigao visoku tačnost na test skupu.
Kroz vizuelizaciju važnosti parametara, dokazano je da povećanje kompleksnosti modela (npr. broj stabala) poboljšava F1-score, dok su detaljne matrice konfuzije potvrdile visoku specifičnost modela u identifikaciji zdravih pacijenata.

Weights & Biases značajno olakšava eksperimentalni rad jer omogućava brzo poređenje različitih modela i konfiguracija.


### Poredjenje sa MLFlow

Weights & Biases je Cloud-native (SaaS) alat. Nakon logovanja i pokretanja koda, rezultati su odmah vidljivi na sajtu. Nema potrebe za podešavanjem baze podataka ili servera. MLflow je self-hosted (otvorenog koda). Da bi se videli rezultati, potrebno je samostalno podići MLflow server i povezati ga sa bazom podataka i skladištem za fajlove. 

Weights & Biases se fokusira na eksplorativnoj analizi podataka i hiperparametara. Generiše visoko interaktivne vizuelizacije koje olakšavaju poredjenje performansi između različitih iteracija modela. MLflow se fokusira na standardizaciji toka rada (workflow) i obezbeđivanju konzistentnosti kroz ceo životni ciklus modela.

Glavna razlika je u tome što jr Weights & Biases primarno istraživački alat dizajniran za optimizaciju i eksperimentisanje, dok je MLflow operativni alat fokusiran na sistemsku organizaciju i upravljanje životnim ciklusom gotovih modela.